In [1]:
from textwrap import wrap
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd



In [2]:
#TODO document jupyter
import pickle
import configparser
import sqlite3 as sl
import pandas as pd
import numpy as np

configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)
datasetPathDatabase =  configParser.get('COMMON', 'datasetPathDatabase') + '/dataset.db'

con = sl.connect(datasetPathDatabase)
data = con.execute("SELECT V.ID, V.VIDEO_PATH, V.AGE, " + 
                    "'This person is '|| V.ETHNICITY || '. ' CAPTION_E, " +
				    "'This person is a '|| lower(V.GENDER) || '. ' CAPTION_G, " +
					"A.SPEAKER_EMB, "+ "A.AUDIO_EMB, " +
					"'This person speaks ' || A.LANG || '. ' CAPTION_L, "+
					"F.FACE_PATH, "+
                    "'The face of a person. ' CAPTION  FROM VIDEO V "+
                    "INNER JOIN AUDIO A ON V.ID = A.VIDEO_ID INNER JOIN FACE F ON V.ID = F.VIDEO_ID WHERE AUDIO_PRE = 2 AND FACES_PRE = 1 AND AUDIO_LENGTH = 24 LIMIT 2500 OFFSET 0")
dataGotten = data.fetchall()

pd.set_option('display.max_columns', None)
df = pd.DataFrame(dataGotten,columns = ['ID','VIDEO_PATH','AGE','caption_e','caption_g','SPEAKER_EMB','AUDIO_EMB','caption_l','image_path','caption'])

def speaker_emb_preprocess(speaker_emb2):
    '''
    A function to take a texual promt and convert it into embeddings
    '''
    #if maxlen is None: maxlen = tokenizer.model_max_length
    #inp = tokenizer(prompts, padding="max_length", max_length=maxlen, truncation=True, return_tensors="pt") 
    #return text_encoder(inp.input_ids.to("cuda"))[0].half()

    #speaker_emb2 = speaker_emb2.squeeze()
    speaker_emb2 = pickle.loads(speaker_emb2)
    #print(speaker_emb2.shape)
    #print(speaker_emb2.shape)
    speaker_emb2 = speaker_emb2.squeeze()
    #print(speaker_emb2.shape)
    speaker_emb2 = np.pad(speaker_emb2, (288), 'constant', constant_values=(0))
    #print(speaker_emb2.shape)
    #print(speaker_emb2.shape)
    speaker_emb2 = np.tile(speaker_emb2, (1, 1))
    #print(speaker_emb2.shape)

    speaker_emb2 = np.array(speaker_emb2).tolist()
    
    
    #speaker_emb2 = torch.from_numpy(speaker_emb2).type(torch.FloatTensor)
    
    #return speaker_emb2.to("cuda").half()
    return speaker_emb2


def audio_emb_preprocess2(speaker_emb2):
    speaker_emb2 = pickle.loads(speaker_emb2)
    a = speaker_emb2 #np.zeros(shape=(24, 512))
    b = np.zeros(shape=(a.shape[0], 768-a.shape[1]))
    c = np.zeros(shape=(50-a.shape[0], 768))
    arr = np.concatenate((a, b), axis=1)
    arr = np.concatenate((arr, c), axis=0)
    speaker_emb2 = np.array(arr).tolist()
    return speaker_emb2

from PIL import Image
def getImage(face_path):
    im = Image.open(face_path)
    im.load() # required for png.split()

    im2 = Image.new("RGB", im.size, (255, 255, 255))
    im2.paste(im, mask=im.split()[3]) # 3 is the alpha channel
    im3 = np.array(im2)
    im4 = np.rollaxis(im3,2)
    #im4 = torch.from_numpy(im4).type(torch.FloatTensor)
    
    #return im4.to("cuda").half()
    return im4

In [3]:
#for index, row in df.iterrows():
#    df.loc[index,"caption"] = speaker_emb_preprocess(df.loc[index,'SPEAKER_EMB'])
#    print(index)

In [4]:
df3 = df[["image_path","caption","caption_e","caption_g","caption_l"]]

In [5]:
df3

,image_path,caption,caption_e,caption_g,caption_l
0,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,This person is white.,This person is a woman.,This person speaks English.
1,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,This person is white.,This person is a woman.,This person speaks English.
2,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,This person is white.,This person is a woman.,This person speaks English.
3,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,This person is asian.,This person is a woman.,This person speaks English.
4,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,This person is asian.,This person is a woman.,This person speaks English.
...,...,...,...,...,...
2495,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,This person is black.,This person is a man.,This person speaks Esperanto.
2496,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,This person is black.,This person is a man.,This person speaks Esperanto.
2497,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,This person is black.,This person is a man.,This person speaks Esperanto.
2498,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,This person is black.,This person is a man.,This person speaks Esperanto.


In [6]:
df3.isnull().values.any()

True

In [7]:
df3.isnull().any(),

(image_path    False
 caption       False
 caption_e      True
 caption_g      True
 caption_l     False
 dtype: bool,)

In [8]:


df3 = df3.fillna('')
df3.isnull().any()

image_path    False
caption       False
caption_e     False
caption_g     False
caption_l     False
dtype: bool

In [9]:
import random

In [10]:
df3['caption_e'] = df3['caption_e'].apply(lambda x: x if random.random() < 0.2 else '')
df3['caption_g'] = df3['caption_g'].apply(lambda x: x if random.random() < 0.2 else '')
df3['caption_l'] = df3['caption_l'].apply(lambda x: x if random.random() < 0.2 else '')

In [11]:
df3

,image_path,caption,caption_e,caption_g,caption_l
0,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,,,
1,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,,This person is a woman.,
2,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,,,This person speaks English.
3,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,,,
4,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,This person is asian.,,
...,...,...,...,...,...
2495,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,,,
2496,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,,,
2497,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,,This person is a man.,This person speaks Esperanto.
2498,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,,,


In [12]:
df3['caption'] = df3['caption'] + df3['caption_e'] + df3['caption_g'] + df3['caption_l']

In [13]:
df3

,image_path,caption,caption_e,caption_g,caption_l
0,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,,,
1,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person. This person is a woman.,,This person is a woman.,
2,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person. This person speaks Engli...,,,This person speaks English.
3,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,,,
4,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person. This person is asian.,This person is asian.,,
...,...,...,...,...,...
2495,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,,,
2496,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,,,
2497,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person. This person is a man. Th...,,This person is a man.,This person speaks Esperanto.
2498,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,,,


In [14]:
df3 = df3[["image_path","caption"]]

In [15]:
df3

,image_path,caption
0,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.
1,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person. This person is a woman.
2,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person. This person speaks Engli...
3,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.
4,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person. This person is asian.
...,...,...
2495,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.
2496,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.
2497,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person. This person is a man. Th...
2498,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.


In [16]:
data_frame = df3
data_frame.head()

,image_path,caption
0,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.
1,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person. This person is a woman.
2,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person. This person speaks Engli...
3,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.
4,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person. This person is asian.


In [19]:
data_frame.dtypes

image_path    object
caption       object
dtype: object

In [20]:
data_frame

,image_path,caption
0,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.
1,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person. This person is a woman.
2,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person. This person speaks Engli...
3,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.
4,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person. This person is asian.
...,...,...
2495,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.
2496,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.
2497,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person. This person is a man. Th...
2498,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.


In [21]:
data_frame['SPEAKER_EMB'] = df['SPEAKER_EMB']
data_frame

/tmp/ipykernel_18884/433991658.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame['SPEAKER_EMB'] = df['SPEAKER_EMB']


,image_path,caption,SPEAKER_EMB
0,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...
1,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person. This person is a woman.,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...
2,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person. This person speaks Engli...,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...
3,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...
4,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person. This person is asian.,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...
...,...,...,...
2495,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...
2496,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...
2497,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person. This person is a man. Th...,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...
2498,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...


In [22]:
data_frame['AUDIO_EMB'] = df['AUDIO_EMB']
data_frame

/tmp/ipykernel_18884/1158286622.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame['AUDIO_EMB'] = df['AUDIO_EMB']


,image_path,caption,SPEAKER_EMB,AUDIO_EMB
0,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x84\x00\x00\x00\x00\x00\x00\x00...
1,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person. This person is a woman.,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x84\x00\x00\x00\x00\x00\x00\x00...
2,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person. This person speaks Engli...,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x84\x00\x00\x00\x00\x00\x00\x00...
3,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x84\x00\x00\x00\x00\x00\x00\x00...
4,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person. This person is asian.,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x84\x00\x00\x00\x00\x00\x00\x00...
...,...,...,...,...
2495,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x84\x00\x00\x00\x00\x00\x00\x00...
2496,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x84\x00\x00\x00\x00\x00\x00\x00...
2497,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person. This person is a man. Th...,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x84\x00\x00\x00\x00\x00\x00\x00...
2498,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x84\x00\x00\x00\x00\x00\x00\x00...


In [23]:
pd.options.mode.chained_assignment = None

for index, row in data_frame.iterrows():
    x = speaker_emb_preprocess(data_frame.loc[index,"SPEAKER_EMB"])
    x = [x]
    data_frame.loc[index,"SPEAKER_EMB"] = x




In [24]:
#CHECK
for index, row in data_frame.iterrows():
    x = audio_emb_preprocess2(data_frame.loc[index,"AUDIO_EMB"])
    x = [x]
    #AADFS = AADFS
    data_frame.loc[index,"AUDIO_EMB"] = x

In [25]:
data_frame

,image_path,caption,SPEAKER_EMB,AUDIO_EMB
0,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[2.2157957553863525, 1.0583380460739136, 2.3..."
1,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person. This person is a woman.,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[2.2157957553863525, 1.0583380460739136, 2.3..."
2,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person. This person speaks Engli...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[2.2157957553863525, 1.0583380460739136, 2.3..."
3,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[2.2157957553863525, 1.699802041053772, 2.35..."
4,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person. This person is asian.,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[2.2157957553863525, 1.699802041053772, 2.35..."
...,...,...,...,...
2495,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[2.2157957553863525, 1.3186947107315063, 2.3..."
2496,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[2.2157957553863525, 1.3186947107315063, 2.3..."
2497,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person. This person is a man. Th...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[2.2157957553863525, 1.3359346389770508, 2.3..."
2498,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[2.2157957553863525, 1.3359346389770508, 2.3..."


In [26]:
data_frame

,image_path,caption,SPEAKER_EMB,AUDIO_EMB
0,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[2.2157957553863525, 1.0583380460739136, 2.3..."
1,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person. This person is a woman.,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[2.2157957553863525, 1.0583380460739136, 2.3..."
2,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person. This person speaks Engli...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[2.2157957553863525, 1.0583380460739136, 2.3..."
3,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[2.2157957553863525, 1.699802041053772, 2.35..."
4,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person. This person is asian.,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[2.2157957553863525, 1.699802041053772, 2.35..."
...,...,...,...,...
2495,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[2.2157957553863525, 1.3186947107315063, 2.3..."
2496,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[2.2157957553863525, 1.3186947107315063, 2.3..."
2497,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person. This person is a man. Th...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[2.2157957553863525, 1.3359346389770508, 2.3..."
2498,/media/gamal/Passport/Datasets/VoxCeleb2/Voxce...,The face of a person.,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[2.2157957553863525, 1.3359346389770508, 2.3..."


In [27]:
import torch
from imagen_pytorch import Unet, Imagen

# unet for imagen

unet1 = Unet(
    dim = 32,
    cond_dim = 512,
    dim_mults = (1, 2, 4, 8),
    num_resnet_blocks = 3,
    layer_attns = (False, True, True, True),
    layer_cross_attns = (False, True, True, True)
)

unet2 = Unet(
    dim = 32,
    cond_dim = 512,
    dim_mults = (1, 2, 4, 8),
    num_resnet_blocks = (2, 4, 8, 8),
    layer_attns = (False, False, False, True),
    layer_cross_attns = (False, False, False, True)
)

# imagen, which contains the unets above (base unet and super resoluting ones)

imagen = Imagen(
    unets = (unet1, unet2),
    image_sizes = (128, 256),
    timesteps = 1000,
    cond_drop_prob = 0.1
).cuda()

# mock images (get a lot of this) and text encodings from large T5

text_embeds = torch.randn(4, 256, 768).cuda()
images = torch.randn(4, 3, 256, 256).cuda()

# feed images into imagen, training each unet in the cascade

for i in (1, 2):
    loss = imagen(images, text_embeds = text_embeds, unet_number = i)
    loss.backward()

# do the above for many many many many steps
# now you can sample an image based on the text embeddings from the cascading ddpm

images = imagen.sample(texts = [
    'a whale breaching from afar',
    'young girl blowing out candles on her birthday cake',
    'fireworks with blue and green sparkles'
], cond_scale = 3.)

images.shape # (3, 3, 256, 256)

imagen.

/home/gamal/anaconda3/envs/ds2f_m_i/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-25 12:07:48.869429: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 12:07:48.919883: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


The base dimension of your u-net should ideally be no smaller than 128, as recommended by a professional DDPM trainer https://nonint.com/2022/05/04/friends-dont-let-friends-train-small-diffusion-models/


sampling loop time step: 100%|██████████| 1000/1000 [02:40<00:00,  6.25it/s]
2it [06:01, 180.81s/it]


torch.Size([3, 3, 256, 256])